In [ ]:
import xmltodict, json
import requests
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv("ebay_api_key.env") #hide production key in environment file



APP_ID = os.getenv("EBAY_PROD_ID")


## URL 
HTTP call url to send the Production Client API. The Payload will modify the url below with a ? followed by an % and key value pairs. This is the most common and recommended way that ebay suggests users to send a request to the Finding API.

## HTTP Headers
Required headers to return a 200 response code. The top header is AuthnAuth security. As there is no need for a user token and the only interface is with my app, the only security required is for the app. The second row is the call be sent to the Finding API, findItemsAdvanced. ## THIS SHOULD BE A LINK DOG.   The third formats the respons as JSON.

## Payload
Modifies HTTP call. Search for knife in the Collectible-Modern-Factory-Manufactured-Folding-Knives that is of the Used Condition. This request returns 100 pages on page 1. 

In [ ]:
url = "https://svcs.ebay.com/services/search/FindingService/v1" #url fir Finding API


headers = {"X-EBAY-SOA-SECURITY-APPNAME": APP_ID, #production client_id
           "X-EBAY-SOA-OPERATION-NAME": "findItemsAdvanced", #call_id
           "X-EBAY-API-RESPONSE-ENCODING": "JSON" #format respons as JSON
          }



payload = {"keywords":"knife", #knife in search bar
           "categoryId":"48818", #Collectible-Modern-Factory-Manufactured-Folding-Knives
          "itemFilter.name":"Condition", #item filter, used condition
          "itemFilter.value":"Used",
          "paginationInput.entriesPerPage":100, #pagination 
          "paginationInput.pageNumber":1
          }

In [ ]:
r = requests.get(url, headers=headers, params=payload)
json_dict_100knivesp1 = xmltodict.parse(r.text)
knives_data_list_of_dicts = json_dict_100knivesp1['findItemsAdvancedResponse']['searchResult']['item']
total_pages = int(json_dict_100knivesp1['findItemsAdvancedResponse']['paginationOutput']['totalPages'])

In [ ]:
#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['itemId', 'title', 'galleryURL', 
        'viewItemURL', 'postalCode', 'location', 
        'country', 'shippingInfo', 'sellingStatus', 
        'listingInfo', 'returnsAccepted', 'condition'
       ]
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
    
       
        # Add to list if all values are present
        if all(prepared_data.values()):
            results.append(prepared_data)
    
    return results

In [ ]:
#organize call with function
prepared_knives = prepare_data(knives_data_list_of_dicts)

In [ ]:
# Create an empty list for the full prepared dataset
full_dataset = []

for page in range(total_pages):
    # Add or update the "offset" key-value pair in url_params
    payload["paginationInput.pageNumber"] = page
    
    # Make the query and get the response
    response = requests.get(url, headers=headers, params=payload)
    
    # Get the response body in JSON format
    response_json = xmltodict.parse(r.text)
    
    # Get the list of businesses from the response_json
    knives_data_list_of_dicts = response_json['findItemsAdvancedResponse']['searchResult']['item']
    
    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(knives_data_list_of_dicts)
    
    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df = pd.DataFrame(full_dataset)

In [ ]:
#Create row for converted Price of Knives in US dollars
price_list = []
for row in full_dataset:
    price_list.append(row['sellingStatus']['convertedCurrentPrice']['#text'])
    
#Create row  for Condition of Knife (should be used)    
condition_list = []
for row in full_dataset_kershaw:
    condition_list.append(row['condition']['conditionDisplayName'])
    
df["price_in_US"] = price_list

df["condition"] = condition_list

df.to_csv("full_datase.csv", index=False)